In [22]:

args = {
    "task": "sentiment+stance",
    "layer": 6,  # 这里假设LAYER的值为6，实际可以根据循环中的值动态设置
    "LLM": "gpt2-small",
    "seed": 42,
    "data_size": -1,
    "device": "cuda",
    "alpha": 20,
    "method": "val_mul",
    "topk_cnt": None,  
    "batch_size": 32,
    "source": None,
    "target": None,
    "prompt_source": "neg",
    "prompt_data_size": 500,
    "mean_type": "dif_mean",
    "steer_type": "last",
    "output_dir": "./results/linear_combination",
    "dataset_path": "/home/ckqsudo/code2024/0dataset/baseline-acl/data/sentiment/sst5",
    "prompt_path": "/home/ckqsudo/code2024/0dataset/baseline-acl/data/prompts/sentiment_prompts-10k",
    "env_path": "/home/ckqsudo/code2024/CKQ_ACL2024/Control_Infer/SAE-simple/.env",
    "save_no_steer": 0,
    "debug": 0,
    "use_cache": 0,
    "repeat_num": 1,
    "gen_batch_size": 16,
    "example_prompt": "But the lack of financial aid would| I feel",
    "temperature": 0.9,  # 新增参数
    "top_p": 0.3,        # 新增参数
    "freq_penalty": 1.0, # 新增参数
}
args["use_cache"]=1
from argparse import Namespace

# 将字典转换为 Namespace 对象
args = Namespace(**args)
sampling_kwargs = {
    "temperature": args.temperature,
    "top_p": args.top_p,
    "freq_penalty": args.freq_penalty,
}

EXAMPLE_PROMPT_LIST=str(args.example_prompt).split("|")
assert isinstance(EXAMPLE_PROMPT_LIST,list) and isinstance(EXAMPLE_PROMPT_LIST[0],str),"EXAMPLE_PROMPT_LIST必须是list[str]"
assert args.example_prompt!="","输入测试prompt"
TASK =args.task
STEER_TYPE=args.steer_type
SOURCE=args.source
TARGET=args.target
# 调整样本正负性在这里调整 从负样本到正样本还是从正样本()到负样本
# pos 代表积极情绪
# neg 代表消极情绪
ALPHA=args.alpha
MAX_NEW_TOKENS=50

SAVE_NO_STEER=args.save_no_steer
DATA_SIZE=args.data_size

In [23]:
stance_path_dir="/home/ckqsudo/code2024/CKQ_ACL2024/Control_Infer/SAE-simple/src/results/stance/gpt2-small_debate_layer_6_datasize_ALL_batchsize32_topK_100/"
# steer_info_cache_of_gpt2-small_l6.pkl
cache_file="steer_info_cache_of_gpt2-small_l6.pkl"
from utils import load_or_cache_steer_info
stance_steer_info=load_or_cache_steer_info(
    CACHE_DIR=stance_path_dir,
    args=args,
    cache_filename=cache_file, 
    compute_func=None)


2025-02-16 13:39:44,793 [INFO] 从缓存 /home/ckqsudo/code2024/CKQ_ACL2024/Control_Infer/SAE-simple/src/results/stance/gpt2-small_debate_layer_6_datasize_ALL_batchsize32_topK_100/steer_info_cache_of_gpt2-small_l6.pkl 中加载 steer_info


In [24]:
stance_steer_info

{'pos': {'latent_value_mean': tensor([5.8366, 4.3402, 1.1432,  ..., 0.7189, 3.2974, 0.0000], device='cuda:0'),
  'latent_frequency': tensor([ 4., 10.,  1.,  ..., 24.,  5.,  0.], device='cuda:0')},
 'neg': {'latent_value_mean': tensor([2.3668, 1.9401, 6.9930,  ..., 0.8718, 0.0000, 0.0000], device='cuda:0'),
  'latent_frequency': tensor([2., 1., 2.,  ..., 9., 0., 0.], device='cuda:0')}}

In [25]:
sentiment_path_dir="/home/ckqsudo/code2024/CKQ_ACL2024/Control_Infer/SAE-simple/src/results/sentiment/gpt2-small_sentiment_layer_6_datasize_ALL_batchsize32_topK_100"
cache_file="steer_info_cache_of_gpt2-small_l6.pkl"

from utils import load_or_cache_steer_info
senti_steer_info=load_or_cache_steer_info(
    CACHE_DIR=sentiment_path_dir,
    args=args,
    cache_filename=cache_file, 
    compute_func=None)

2025-02-16 13:39:52,875 [INFO] 从缓存 /home/ckqsudo/code2024/CKQ_ACL2024/Control_Infer/SAE-simple/src/results/sentiment/gpt2-small_sentiment_layer_6_datasize_ALL_batchsize32_topK_100/steer_info_cache_of_gpt2-small_l6.pkl 中加载 steer_info


In [26]:
senti_steer_info

{'pos': {'latent_value_mean': tensor([ 5.1003, 13.6636,  8.1277,  ...,  0.7722,  0.9422,  0.0000],
         device='cuda:0'),
  'latent_frequency': tensor([17.,  8.,  1.,  ..., 62., 44.,  0.], device='cuda:0')},
 'neg': {'latent_value_mean': tensor([6.9864, 8.3967, 7.4535,  ..., 0.8572, 1.2406, 0.4634], device='cuda:0'),
  'latent_frequency': tensor([13.,  4.,  1.,  ..., 57., 28.,  4.], device='cuda:0')}}

In [27]:
import torch

In [55]:
senti_target="neg"
senti_source="pos"
stance_target="pos"
stance_source="neg"

In [56]:
# steer_info=

senti_steer_info[f"dif_{senti_target}-{senti_source}_relu"]={"latent_frequency":torch.relu(senti_steer_info[senti_target]["latent_frequency"]-senti_steer_info[senti_source]["latent_frequency"]),"latent_value_mean":torch.relu(senti_steer_info[senti_target]["latent_value_mean"]-senti_steer_info[senti_source]["latent_value_mean"]),"ID":"pos<-neg"}
# 

# 积极是赞同，消极是不赞同，那pos-
target=oppo="neg"
source=supp="pos"
stance_steer_info[f"dif_{target}-{source}_relu"]={"latent_frequency":torch.relu(stance_steer_info[target]["latent_frequency"]-stance_steer_info[source]["latent_frequency"]),"latent_value_mean":torch.relu(stance_steer_info[target]["latent_value_mean"]-stance_steer_info[source]["latent_value_mean"]),"ID":"oppo<-supp"}
target="pos"
source="neg"
stance_steer_info[f"dif_{target}-{source}_relu"]={"latent_frequency":torch.relu(stance_steer_info[target]["latent_frequency"]-stance_steer_info[source]["latent_frequency"]),"latent_value_mean":torch.relu(stance_steer_info[target]["latent_value_mean"]-stance_steer_info[source]["latent_value_mean"]),"ID":"oppo<-supp"}

"""
nz_cnt: 神经元被激活的次数
nz_mean: 神经元被激活后的平均值
nz_mean_pos: 正样本神经元被激活后的平均值
"""

# top_k=args.topk_cnt

'\nnz_cnt: 神经元被激活的次数\nnz_mean: 神经元被激活后的平均值\nnz_mean_pos: 正样本神经元被激活后的平均值\n'

In [57]:
_,senti_topk_indices=torch.topk(senti_steer_info[f"dif_{senti_target}-{senti_source}_relu"]["latent_frequency"],100)

In [58]:
_,stance_topk_indices=torch.topk(stance_steer_info[f"dif_{stance_target}-{stance_source}_relu"]["latent_frequency"],50)

In [59]:

mask = torch.isin(senti_topk_indices, stance_topk_indices)

In [60]:
# 统计大于 0 的
count = torch.sum(mask).item()
# 反对立场转向与积极情感转向的神经元重合率只有5，积极立场转向与积极情感转向的重合率有28%
count

1

In [61]:
import torch
prefer="stance"
import torch

# 初始化张量
combine_latent_mean = torch.zeros(senti_steer_info[f"dif_{senti_target}-{senti_source}_relu"]["latent_value_mean"].shape,device=args.device)

# 合并两个张量并获取唯一元素及计数
combined_indices = torch.cat((senti_topk_indices, stance_topk_indices))
unique_indices, counts = torch.unique(combined_indices, return_counts=True)

# 筛选出重复元素
duplicate_indices = unique_indices[counts > 1]
# 筛选出不重复元素
unique_indices = unique_indices[counts == 1]

# 不重复元素的更新
for idx in unique_indices:
    if idx in senti_topk_indices and idx not in stance_topk_indices:
        combine_latent_mean[idx] = senti_steer_info[f"dif_{senti_target}-{senti_source}_relu"]["latent_value_mean"][idx]
    elif idx in stance_topk_indices and idx not in senti_topk_indices:
        combine_latent_mean[idx] = stance_steer_info[f"dif_{stance_target}-{stance_source}_relu"]["latent_value_mean"][idx]

# 处理重复元素，根据偏好选择
if prefer == "stance":
    combine_latent_mean[duplicate_indices] = stance_steer_info[f"dif_{stance_target}-{stance_source}_relu"]["latent_value_mean"][duplicate_indices]
elif prefer == "senti":
    combine_latent_mean[duplicate_indices] = senti_steer_info[f"dif_{senti_target}-{senti_source}_relu"]["latent_value_mean"][duplicate_indices]
else:
    raise ValueError("Unsupported preference")

# 计算最终的 steer_indices
steer_indices = torch.cat((unique_indices, duplicate_indices))

combine_latent_mean

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')

In [62]:
unique_indices.shape, duplicate_indices.shape

(torch.Size([148]), torch.Size([1]))

In [63]:
from torch import Tensor
from transformer_lens import HookedTransformer
from log import setup_logging
import logging
from sae_lens import SAE
import os
CACHE_DIR=os.path.join(args.output_dir,f"{args.LLM}_{args.task}_layer_{args.layer}_datasize_{args.data_size}_batchsize{args.batch_size}_topK_{args.topk_cnt}")
OUTPUT_DIR=os.path.join(CACHE_DIR,f"alpha_{args.alpha}_from_{args.source}_to_{args.target}_prompt_{args.prompt_source}_mean_{args.mean_type}_steertype_{args.steer_type}_device_{args.device}")

setup_logging(OUTPUT_DIR)
if "gpt2-small" in args.LLM:
    logging.info(f"Loading model: {args.LLM} SAE gpt2-small-res-jb")
    sae, cfg_dict, sparsity = SAE.from_pretrained(
        release=f"{args.LLM}-res-jb",
        sae_id=f"blocks.{args.layer}.hook_resid_pre",
        device=args.device
    )
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    # 设置填充标记为 EOS token
    tokenizer.pad_token = tokenizer.eos_token
    model = HookedTransformer.from_pretrained(args.LLM, device=args.device,tokenizer=tokenizer)
    # tokenizer = AutoTokenizer.from_pretrained(args.LLM)
else:
    raise ValueError("No Supported")

2025-02-16 14:19:34,671 [INFO] Logging initialized. Logs will be saved to ./results/linear_combination/gpt2-small_sentiment+stance_layer_6_datasize_-1_batchsize32_topK_None/alpha_20_from_None_to_None_prompt_neg_mean_dif_mean_steertype_last_device_cuda/execution.log
2025-02-16 14:19:34,672 [INFO] Loading model: gpt2-small SAE gpt2-small-res-jb


Loaded pretrained model gpt2-small into HookedTransformer


In [64]:
def SAE_decoding(sae: SAE, indices: Tensor, latent_value_mean: Tensor, method: str = "val_mul",is_norm:int=0) -> Tensor:
    assert is_norm in [0,1] and method in ["val_mul"], "Invalid arguments"
    delta_h = torch.zeros(sae.W_dec.shape[1], device=sae.W_dec.device)
    for idx in indices:
        delta_h += latent_value_mean[idx].item() * sae.W_dec[idx]
    return delta_h

if args.mean_type=="dif_mean":
    delta_h=SAE_decoding(sae,indices=steer_indices,latent_value_mean=combine_latent_mean,method="val_mul",is_norm=0)
elif args.mean_type=="tar_mean":
    raise ValueError("Unsupported")
else:
    raise ValueError("Unsupported")

In [43]:
EXAMPLE_PROMPT_LIST=["But the lack of financial aid would","The passage of the AI Act will"]
ALPHA=50

In [65]:
from apply_control import run_generate
logging.info("Generating texts **without** steering... ")
with torch.no_grad():
    generated_texts_no_steer = run_generate(
        prompts=EXAMPLE_PROMPT_LIST, 
        sampling_kwargs=sampling_kwargs,
        sae=sae,
        model=model,
        tokenizer=tokenizer,
        MAX_NEW_TOKENS=MAX_NEW_TOKENS,
        repeat_num=3,
        steer_type="",
        steer_on=False,
        alpha=0,
        delta_h=None,
        show_res=True)
logging.info("干预之后的结果")
# bef,aft=args.steer.split("-")
# logging.info(f"干预方向{SOURCE}->{TARGET},礼貌任务下，neg=impolite，情感任务下 pos=积极情感")
logging.info("** Generating texts with steering... Target **")
# logging.info(f"form {SOURCE} to {TARGET}")
generated_texts_with_steer = run_generate(
    prompts=EXAMPLE_PROMPT_LIST, 
    sampling_kwargs=sampling_kwargs,
    sae=sae,
    model=model,
    tokenizer=tokenizer,
    MAX_NEW_TOKENS=MAX_NEW_TOKENS,
    repeat_num=3,
    steer_on=True,
    alpha=ALPHA,
    steer_type=STEER_TYPE,
    delta_h=delta_h,
    show_res=True)

2025-02-16 14:22:42,027 [INFO] Generating texts **without** steering... 
2025-02-16 14:22:42,033 [INFO] 无干预


  0%|          | 0/50 [00:00<?, ?it/s]

2025-02-16 14:22:43,844 [INFO] 当前批次共处理2个prompt
2025-02-16 14:22:43,848 [INFO] Prompt 1: |But the lack of financial aid would|
2025-02-16 14:22:43,850 [INFO] 生成 1: | be a major blow to the country's economy, which is struggling to recover from its worst recession in decades.

The IMF has warned that the country's growth could be slowed by an increase in exports and by an increase in imports.

|
2025-02-16 14:22:43,850 [INFO] 生成 2: | have made it difficult for many to afford the medical care they need.

"We're not going to be able to pay for this," said Mr. Carnevale, who has been in a wheelchair since he was 10 years old. "|
2025-02-16 14:22:43,851 [INFO] 生成 3: | mean that the United States would have to rely on foreign aid, not just from countries like China.

"The United States has a long history of using international aid to help people," said Thomas R. Hoeven, a professor at the|
2025-02-16 14:22:43,851 [INFO] Prompt 2: |The passage of the AI Act will|
2025-02-16 14:22:43,851 [INFO]

  0%|          | 0/50 [00:00<?, ?it/s]

2025-02-16 14:22:45,737 [INFO] 当前批次共处理2个prompt
2025-02-16 14:22:45,738 [INFO] Prompt 1: |But the lack of financial aid would|
2025-02-16 14:22:45,738 [INFO] 生成 1: | be a big blow to our nation's ability to provide for its citizens.

"The federal government is not responsible for the actions of its own employees," said John M. Campbell, a senior fellow at the Center on Budget and Policy Priorities|
2025-02-16 14:22:45,738 [INFO] 生成 2: | be a major blow to the city's reputation as a safe haven for immigrants.

The area has been plagued by crime since the 1980s, and many residents have lost their jobs due to illegal immigration. The city's unemployment rate is at its|
2025-02-16 14:22:45,738 [INFO] 生成 3: | have been a mistake.

The New York Times reported that Trump's campaign manager, Paul Manafort, had "paid $200,000 to a Russian lawyer who worked for the Kremlin to help elect Hillary Clinton." The Washington Post reported that Manafort had|
2025-02-16 14:22:45,739 [INFO] Prompt 2: |The

In [16]:
EXAMPLE_PROMPT_LIST=["I agree with the act","The act of AI will"]

In [ ]:
import jsonlines
import tqdm
import copy
# 
TASK="sentiment"
def eval_on_full_data():
    """跑全量实验
    Raises:
        NotImplementedError: _description_
    """
    logging.info("Running on full data")
    
    if TASK=="sentiment":
        logging.info("Out of Domain: Calculate at A dataset, Evaluate at B dataset")
        from data_preprocess import load_and_prepare_sentiment_prompts
        prompts=load_and_prepare_sentiment_prompts(prompt_path=args.prompt_path,task=TASK)
    else:
        raise NotImplementedError("No Supported Task")    
    assert "neu" in prompts,"prompt steer source (pos/neg) not in prompts, please check the data_preprocess section"
    if args.prompt_source!="":
        logging.info(f"prompt的极性是{args.prompt_source}")
        prompts=prompts[args.prompt_source]
    else:
        prompts=prompts[SOURCE]
    
    param={**vars(args),**sampling_kwargs,"max_new_tokens":50,"steer":f"from {SOURCE} to {TARGET}"}
    param["alpha_recheck"]=ALPHA
    logging.info(f"Running with alpha: {ALPHA}")
    logging.info(f"Running with prompt_type: "+str(param["steer"]))

    
    # 打开文件（模式为追加模式 'a'）
    with jsonlines.open(os.path.join(OUTPUT_DIR,"params.jsonl"), mode='w') as writer:
        writer.write(param)  # 逐行写入
    if args.prompt_data_size!=-1:# 用于小批量网格搜索
        prompts = prompts.select(range(args.prompt_data_size))  # Correct slicing for datase
        logging.warning("截取prompt_datasize"+str(len(prompts)))
    
    # 新增批次大小参数（假设从args获取）
    GEN_BATCH_SIZE = args.gen_batch_size

    with jsonlines.open(os.path.join(OUTPUT_DIR,f"no_steer_gen_res_{TASK}.jsonl"), mode='w') as no_steer_f:
        with jsonlines.open(os.path.join(OUTPUT_DIR,f"steer_gen_res_{TASK}.jsonl"), mode='w') as steer_f: 
            # 分批次处理prompts
            for batch_idx in range(0, len(prompts), GEN_BATCH_SIZE):
                # batch = prompts[batch_idx:batch_idx+GEN_BATCH_SIZE]
                batch = prompts.select(range(batch_idx, min(batch_idx + GEN_BATCH_SIZE, len(prompts))))
                batch_prompts = [item["prompt"]["text"] for item in batch]
                
                # 无转向生成
                if SAVE_NO_STEER == 1:
                    with torch.no_grad():
                        no_steer_gen_texts_batch = run_generate(
                            prompts=batch_prompts,  # 传入批次prompts
                            sampling_kwargs=sampling_kwargs,
                            sae=sae,
                            model=model,
                            tokenizer=tokenizer,
                            MAX_NEW_TOKENS=MAX_NEW_TOKENS,
                            repeat_num=args.repeat_num,
                            steer_type="",
                            steer_on=False,
                            alpha=0,
                            delta_h=None,
                            show_res=False
                        )
                    
                    # 写入无转向结果
                    for i, item in enumerate(batch):
                        no_steer_item = copy.deepcopy(item)
                        no_steer_item["generations"] = [{"text": text} for text in no_steer_gen_texts_batch[i]]
                        no_steer_f.write(no_steer_item)

                # 转向生成
                steered_texts_batch = run_generate(
                    prompts=batch_prompts, 
                    sampling_kwargs=sampling_kwargs,
                    sae=sae,
                    model=model,
                    tokenizer=tokenizer,
                    MAX_NEW_TOKENS=MAX_NEW_TOKENS,
                    repeat_num=args.repeat_num,
                    steer_on=True,
                    alpha=ALPHA,
                    steer_type=STEER_TYPE,
                    delta_h=delta_h,
                    show_res=False
                )
                
                # 写入转向结果
                for i, item in enumerate(batch):# 遍历每个batch中的元素
                    steer_item = copy.deepcopy(item)
                    steer_item["generations"] = [{"text": text} for text in steered_texts_batch[i]]
                    global_idx = batch_idx + i
                    if global_idx % 20 == 0:
                        logging.info(f"{TASK}: from {SOURCE} to {TARGET} prompt_set: {SOURCE}")
                        logging.info(steer_item)
                    steer_f.write(steer_item)

if args.debug==1:
    logging.info(f"debug mode,show example, no full dataset eval")
elif args.debug==0:
    if SAVE_NO_STEER==1:
        logging.info("Provide No Steer Result 提供无干预对照样本")
    eval_on_full_data()
else:
    raise ValueError("debug must be 0 or 1")
# %%
from utils import params_to_dict
params = params_to_dict(args, is_print=True)
with jsonlines.open(os.path.join(OUTPUT_DIR, "params.jsonl"), mode='w') as param_file:
    param_file.write(params)

2025-02-10 15:01:44,409 [INFO] Running on full data
2025-02-10 15:01:44,411 [INFO] Out of Domain: Calculate at A dataset, Evaluate at B dataset
2025-02-10 15:01:44,412 [INFO] Loading prompt_path from /home/ckqsudo/code2024/0dataset/baseline-acl/data/prompts/sentiment_prompts-10k
2025-02-10 15:01:44,448 [INFO] prompt的极性是neg
2025-02-10 15:01:44,449 [INFO] Running with alpha: 100
2025-02-10 15:01:44,450 [INFO] Running with prompt_type: from None to None
2025-02-10 15:01:44,456 [WARNING] 截取prompt_datasize500


DatasetDict({
    neg: Dataset({
        features: ['md5_hash', 'prompt', 'continuation', 'num_positive'],
        num_rows: 2500
    })
    pos: Dataset({
        features: ['md5_hash', 'prompt', 'continuation', 'num_positive'],
        num_rows: 2500
    })
    neu: Dataset({
        features: ['md5_hash', 'prompt', 'continuation', 'num_positive'],
        num_rows: 5000
    })
})


  0%|          | 0/50 [00:00<?, ?it/s]

2025-02-10 15:01:45,504 [INFO] sentiment: from None to None prompt_set: None
2025-02-10 15:01:45,505 [INFO] {'md5_hash': '06daa8998220e9f416c42b8f9c45fda8', 'prompt': {'text': 'It made my hair feel flat and uncooperative'}, 'continuation': {'text': ', though my skin felt soft and relaxed.'}, 'num_positive': 0, 'generations': [{'text': 'It made my hair feel flat and uncooperative against the words we use against our fighting against: our words against the word of God. Against us against the anti-\xad\xad\xad\xad\xad\u200bwords. Against our fighting in every corner of the world against all who are fighting for their freedom from'}]}


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

2025-02-10 15:02:14,188 [INFO] sentiment: from None to None prompt_set: None
2025-02-10 15:02:14,189 [INFO] {'md5_hash': 'd57568ce10f19bc27be29d47c0d77374', 'prompt': {'text': 'The steepest decreases occurred among men who reported that they'}, 'continuation': {'text': 'spent time caring for and palling around with their children.'}, 'num_positive': 0, 'generations': [{'text': 'The steepest decreases occurred among men who reported that they against the name of the brave. Against the name of a man who is fighting against an anti-war, against a fighter against his fighting for our right to fight in the face of this battle. Against his anti-tidal battle against our water'}]}


  0%|          | 0/50 [00:00<?, ?it/s]

2025-02-10 15:02:14,955 [INFO] Show Hyperparameters: 


2025-02-10 15:02:14,956 [INFO]   task: sentiment+stance
2025-02-10 15:02:14,957 [INFO]   layer: 6
2025-02-10 15:02:14,957 [INFO]   LLM: gpt2-small
2025-02-10 15:02:14,958 [INFO]   seed: 42
2025-02-10 15:02:14,959 [INFO]   data_size: -1
2025-02-10 15:02:14,959 [INFO]   device: cuda
2025-02-10 15:02:14,960 [INFO]   alpha: 100
2025-02-10 15:02:14,960 [INFO]   method: val_mul
2025-02-10 15:02:14,961 [INFO]   topk_cnt: None
2025-02-10 15:02:14,962 [INFO]   batch_size: 32
2025-02-10 15:02:14,962 [INFO]   source: None
2025-02-10 15:02:14,963 [INFO]   target: None
2025-02-10 15:02:14,964 [INFO]   prompt_source: neg
2025-02-10 15:02:14,964 [INFO]   prompt_data_size: 500
2025-02-10 15:02:14,965 [INFO]   mean_type: dif_mean
2025-02-10 15:02:14,965 [INFO]   steer_type: last
2025-02-10 15:02:14,966 [INFO]   output_dir: ./results/linear_combination
2025-02-10 15:02:14,967 [INFO]   dataset_path: /home/ckqsudo/code2024/0dataset/baseline-acl/data/

In [ ]:
from datasets import load_dataset
from dotenv import load_dotenv
from openai import OpenAI
import os
import argparse
import os
import re
import json
import einops
import random
from typing import Tuple
from tqdm import tqdm
from log import setup_logging
import logging
import torch
from torch import Tensor
from transformer_lens import HookedTransformer
from sae_lens import SAE
from datasets import load_dataset
import numpy as np
import argparse

from data_preprocess import load_and_prepare_triple_dataset,load_and_prepare_COT_dataset,load_and_prepare_debate_triple_dataset, load_and_prepare_polite_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from utils import load_environment
load_dotenv('/home/ckqsudo/code2024/CKQ_ACL2024/Control_Infer/SAE-simple/.env')
qwen_api_key = os.getenv('DASHSCOPE_API_KEY')
def get_qwen_eval(text):
    client = OpenAI(api_key=qwen_api_key, base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
    response = client.chat.completions.create(
        model="qwen-max",
        messages=[
            {"role": "system", "content": "You are a helpful assistant, you need to help me determine the stance of a given sentence, You need to guess whether this sentence is expressing a positive support for something or a negative opposition. Your answer can only be one of the two words 'support' or 'oppose'."},
            {"role": "user", "content": text},
        ],
        stream=False
    )
    # print(response.choices[0].message.content)
    return response.choices[0].message.content
def load_and_prepare_debate_prompts(prompt_path:str,task:str):
    assert task in ["debate"],"请输入正确的任务"
    prompts = load_dataset(prompt_path)
    sup_train_set = prompts['test'].filter(lambda example: example['label'] == 'support')
    opp_train_set = prompts['test'].filter(lambda example: example['label'] == 'oppose')
    return sup_train_set['text'],opp_train_set['text']


sup_prompt, opp_prompt = load_and_prepare_debate_prompts(prompt_path='/home/ckqsudo/code2024/0dataset/baseline-acl/data/debate/StanceSentences',task='debate')

final_eval_results = []
for text in opp_prompt:
    origin_text = text
    text = text[:30] if len(text) > 30 else text
    generated_text_no_steer = run_generate(
                            prompts=[text],  # 传入批次prompts
                            sampling_kwargs=sampling_kwargs,
                            sae=sae,
                            model=model,
                            tokenizer=tokenizer,
                            MAX_NEW_TOKENS=MAX_NEW_TOKENS,
                            repeat_num=args.repeat_num,
                            steer_type="",
                            steer_on=False,
                            alpha=0,
                            delta_h=None,
                            show_res=False
                        )
                    

    # 转向生成
    generated_text_with_steer = run_generate(
        prompts=[text], 
        sampling_kwargs=sampling_kwargs,
        sae=sae,
        model=model,
        tokenizer=tokenizer,
        MAX_NEW_TOKENS=MAX_NEW_TOKENS,
        repeat_num=args.repeat_num,
        steer_on=True,
        alpha=ALPHA,
        steer_type=STEER_TYPE,
        delta_h=delta_h,
        show_res=False
    )
    # final_eval_results.append({"prompt_text": text,
    #                            "origin_text": origin_text,
    #                           "no_steer_text": generated_text_no_steer[0].replace(text, ''),
    #                           "with_steer_text": generated_text_with_steer[0].replace(text, ''),
    #                           "no_steer_eval": get_qwen_eval(generated_text_no_steer[0].replace(text, '')),
    #                           "with_steer_eval": get_qwen_eval(generated_text_with_steer[0].replace(text, ''))})

def conditional_perplexity(texts, model, tokenizer, device='cuda', eval_target='no_steer'):
    perplexities = []
    goodperplexities = []
    total_nll = 0
    total_tokens = 0
    g = 0
    ct = 0

    for text in tqdm(texts, desc='Evaluating PPL'):
        prompt = text['origin_text']
        prompt_input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
        if not (prompt_input_ids.shape[1] == 1 and prompt_input_ids[0].tolist()[0] == tokenizer.bos_token_id):
            prompt_loss = model(prompt_input_ids, labels=prompt_input_ids)[0] * (prompt_input_ids.shape[1]-1)
        else:
            prompt_loss = 0

        gen = text['no_steer_text'] if eval_target == 'no_steer' else text['with_steer_text']
        full_input_ids = tokenizer.encode(f'{prompt}{gen}', return_tensors='pt').to(device)
        full_loss = model(full_input_ids, labels=full_input_ids)[0] * (full_input_ids.shape[1]-1)
        loss = (full_loss - prompt_loss) / (full_input_ids.shape[1] - prompt_input_ids.shape[1])
        ppl = np.exp(loss.item())
        if ppl < 100:
            goodperplexities.append(ppl)
            g += 1
        if ppl < 1e4:
            perplexities.append(ppl)
        total_nll += (full_loss - prompt_loss).item()
        total_tokens += (full_input_ids.shape[1] - prompt_input_ids.shape[1])

    print(np.nanmean(goodperplexities), len(goodperplexities), len(perplexities), g)
    return np.nanmean(perplexities), np.exp(total_nll/total_tokens)

def distinctness(texts, eval_target='no_steer'):
    dist1, dist2, dist3 = [], [], []
    
    for text in tqdm(texts, desc='Evaluating dist-n'):
        gens = text['no_steer_text'] if eval_target == 'no_steer' else text['with_steer_text']
        unigrams, bigrams, trigrams = set(), set(), set()
        total_words = 0
        o = gens.split(' ')
        total_words += len(o)
        unigrams.update(o)
        for i in range(len(o) - 1):
            bigrams.add(o[i] + '_' + o[i+1])
        for i in range(len(o) - 2):
            trigrams.add(o[i] + '_' + o[i+1] + '_' + o[i+2])
        dist1.append(len(unigrams) / total_words)
        dist2.append(len(bigrams) / total_words)
        dist3.append(len(trigrams) / total_words)

    return np.nanmean(dist1), np.nanmean(dist2), np.nanmean(dist3)


gpt2_path="/home/ckqsudo/code2024/0models/gpt-2-openai/gpt-2-openai"

eval_model = AutoModelForCausalLM.from_pretrained(gpt2_path).to(args.device)
eval_tokenizer = AutoTokenizer.from_pretrained(gpt2_path)

ppl1, total_ppl_no_steer = conditional_perplexity(final_eval_results, eval_model, eval_tokenizer, device='cuda', eval_target='no_steer')
ppl2, total_ppl_with_steer = conditional_perplexity(final_eval_results, eval_model, eval_tokenizer, device='cuda', eval_target='with_steer')

dist1, dist2, dist3 = distinctness(final_eval_results, eval_target='with_steer')


sup_num_no_steer, sup_num_with_steer = 0,0
for item in final_eval_results:
    if 'support' in item['no_steer_eval']:
        sup_num_no_steer += 1
    if 'support' in item['with_steer_eval']:
        sup_num_with_steer += 1
    
    
with open (f'/home/ckqsudo/code2024/CKQ_ACL2024/Control_Infer/SAE-simple/src/debate_test/debate_eval_results/eval_results_controllm_alpha{args.alpha}_supportrate_from{sup_num_no_steer}_to{sup_num_with_steer}_pplfrom{round(total_ppl_no_steer,2)}_to{round(total_ppl_with_steer,2)}_dist123_{round(dist1,2)}_{round(dist2,2)}_{round(dist3,2)}.json','w') as f:
    json.dump(final_eval_results,f,ensure_ascii=False,indent=4)

2025-02-10 15:27:06,896 [INFO] 无干预


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

AttributeError: 'list' object has no attribute 'replace'